# Rate performance

   * [Comparison new vs old EG ID](#Comparisono-baseline-vs-Manos-EGID)




In [1]:
import sys
sys.path.insert(0, '..')
# sys.path.insert(1, '../python/')
sys.path.insert(1, 'python/')

import ROOT
import root_numpy as rnp
# import python.l1THistos as histos
import math
import uuid
import pandas as pd

from drawingTools import *

# ROOT.enableJSVis()
ROOT.enableJSVis()

#from drawingTools import *

normalized_histos = list()

Welcome to JupyROOT 6.14/02


In [14]:
# %load python/drawingTools.py
# %load drawingTools.py
import ROOT
import math
import uuid
import pandas as pd


# some useful globals, mainly to deal with ROOT idiosyncrasies
c_idx = 0
p_idx = 0
colors = range(1, 12)
colors = [1, 632-4, 416+1, 600-3, 616+1, 432-3]


marker_styles = [8, 21, 22, 23, 33]
stuff = []
f_idx = 0

ROOT.gStyle.SetOptTitle(False)
ROOT.gStyle.SetPadBottomMargin(0.13)
ROOT.gStyle.SetPadLeftMargin(0.13)
ROOT.gStyle.SetPadRightMargin(0.13)
ROOT.gStyle.SetOptStat(False)

# ROOT.gStyle.SetCanvasBorderMode(0)
# ROOT.gStyle.SetCanvasColor(0)


# def DrawPrelimLabel(canvas):
#     canvas.cd()
#     tex = ROOT.TLatex()
#     global stuff
#     stuff.append(tex)
#     tex.SetTextSize(0.03)
#     tex.DrawLatexNDC(0.13,0.91,"#scale[1.5]{CMS} #scale[1.]{Phase-2 Simulation}")
#     tex.DrawLatexNDC(0.49,0.91,"14TeV, 7.5#times10^{34}cm^{-2}s^{-1}, 200 PU")

#     tex.Draw("same");
#     return

def SaveCanvas(canvas, name):
    canvas.cd()
    canvas.SaveAs(name+'.pdf')


# void SaveCanvas(TCanvas* c, TString PlotName = "myPlotName")
# {
#   c->cd();
#   c->SaveAs(PlotName + ".pdf");
#   c->SaveAs(PlotName + ".root");

#   return;
# }



def getText(text, ndc_x, ndc_y):
    global stuff
    rtext = ROOT.TLatex(ndc_x, ndc_y, text)
    stuff.append(rtext)
    rtext.SetNDC(True)
    # rtext.SetTextFont(40)
    rtext.SetTextSize(0.03)
    return rtext


def getLegend(x1=0.7, y1=0.71, x2=0.95, y2=0.85):
    global stuff
    legend = ROOT.TLegend(x1, y1, x2, y2)
    stuff.append(legend)
    legend.SetFillColor(0)
    legend.SetFillStyle(0)
    legend.SetBorderSize(0)
    legend.SetTextSize(0.03)
    return legend


def newCanvas(name=None, title=None, height=600, width=800, xdiv=0, ydiv=0, form=4):
    global c_idx
    if name is None:
        name = 'c_{}'.format(uuid.uuid4().hex[:6])
        c_idx += 1
    if title is None:
        title = name
    # print name, title, width, height
    canvas = ROOT.TCanvas(name, title, width, height)
    if(xdiv*ydiv != 0):
        canvas.Divide(xdiv, ydiv)
    global stuff
    stuff.append(canvas)
    return canvas


def drawAndProfileX(plot2d, miny=None, maxy=None, do_profile=True, options='', text=None):
    global p_idx
    if miny and maxy:
        plot2d.GetYaxis().SetRangeUser(miny, maxy)
    c = newCanvas()
    c.SetGrid(1, 1)
    c.cd()
    plot2d.Draw(options)
    ROOT.gPad.SetGrid(1, 1)
    ROOT.gStyle.SetGridColor(15)

    if do_profile:
        profname = plot2d.GetName()+'_prof_'+str(p_idx)
        p_idx += 1
        firstbin = 1
        lastbin = -1
        prof = plot2d.ProfileX(profname, firstbin, lastbin, 's')
        prof.SetMarkerColor(2)
        prof.SetLineColor(2)
        prof.Draw('same')

    if text:
        rtext = getText(text, 0.15, 0.85)
        rtext.Draw('same')


    c.Draw()

class DrawConfig(object):
    def __init__(self):
        self.do_stats = False
        self.marker_size = 0.5
        self.marker_styles = [8, 21, 22, 23, 33]
        self.colors = [1, 632-4, 416+1, 600-3, 616+1, 432-3]
#         self.canvas_sizes = (800, 600)
        self.canvas_sizes = (600, 600)
        # SetMargin (Float_t left, Float_t right, Float_t bottom, Float_t top)
#         self.canvas_margins = (0.13, 0.3, 0.13, 0.1)
        self.canvas_margins = (0.13, 0.13, 0.13, 0.1)
        self.canvas_margins_div = (0.13, 0.13, 0.13, 0.1)
        self.legend_position = (0.6, 0.7)
        self.legend_size = (0.26, 0.1)
        self.additional_text = []
        self.additional_text_size = 0.03
        return

tdr_config = DrawConfig()
tdr_config.additional_text.append((0.13,0.91,"#scale[1.5]{CMS} #scale[1.]{Phase-2 Simulation}"))
tdr_config.additional_text.append((0.69,0.91,"14TeV, 200 PU"))

rleg_config = DrawConfig()
rleg_config.canvas_sizes = (800, 600)
rleg_config.canvas_margins = (0.13, 0.3, 0.13, 0.1)
rleg_config.legend_position = (0.7, 0.71)
rleg_config.legend_size = (0.25, 0.14)


class DrawMachine(object):
    def __init__(self, config):
        global stuff
        self.config = config
        self.histos = []
        stuff.append(self.histos)
        self.labels = []
        self.overlay = True
        self.canvas = None
        self.legend = None
        return

    def addHistos(self, histograms, labels):
        for hidx,hist in enumerate(histograms):
            histo_class = hist.ClassName()
            if 'TH2' in histo_class or 'TH3' in histo_class:
                self.overlay = False

            # clone the histo
            d_hist = hist.Clone(uuid.uuid4().hex[:6])
#             if 'TGraph' in histo_class:
#                 d_hist.SetTitle(';'+';'.join(hist.GetTitle().split(';')[1:]))
#             else:
            d_hist.SetTitle(';{};{}'.format(hist.GetXaxis().GetTitle(), hist.GetYaxis().GetTitle()))
            # drop the title
            #             d_hist.SetTitle("")
            self.histos.append(d_hist)
            self.labels.append(labels[hidx])
        return


    def drawAdditionalText(self):
        tex = ROOT.TLatex()
        global stuff
        stuff.append(tex)
        tex.SetTextSize(self.config.additional_text_size)
        for txt in self.config.additional_text:
            tex.DrawLatexNDC(txt[0], txt[1], txt[2])
        tex.Draw("same");


    def formatHistos(self):
        for hidx,hist in enumerate(self.histos):
            histo_class = hist.ClassName()
            hist.UseCurrentStyle()

            if 'TGraph' in histo_class:
                hist.SetMarkerSize(self.config.marker_size)
                hist.SetMarkerStyle(self.config.marker_styles[hidx])
                if self.overlay:
                    hist.SetMarkerColor(colors[hidx])
                    hist.SetLineColor(self.config.colors[hidx])

            else:
                hist.SetStats(self.config.do_stats)
                if self.overlay:
                    hist.SetLineColor(self.config.colors[hidx])
        return

    def createCanvas(self, do_ratio=False):
        if self.canvas is not None:
            return

        xdiv = 0
        ydiv = 0

        c_width, c_height = self.config.canvas_sizes

        if not self.overlay:
            xdiv = 2
            ydiv = math.ceil(float(len(self.histos))/2)
            c_width = 1000.
            c_height = 500*ydiv
        else:
            if do_ratio:
                c_height = c_height+200

        self.canvas = newCanvas(name=None,
                                title=None,
                                height=int(c_height),
                                width=int(c_width),
                                xdiv=int(xdiv),
                                ydiv=int(ydiv))
        if self.overlay:
            self.canvas.SetMargin(self.config.canvas_margins[0],
                                  self.config.canvas_margins[1],
                                  self.config.canvas_margins[2],
                                  self.config.canvas_margins[3])
        else:
            self.canvas.SetMargin(self.config.canvas_margins_div[0],
                                  self.config.canvas_margins_div[1],
                                  self.config.canvas_margins_div[2],
                                  self.config.canvas_margins_div[3])
        return

    def createLegend(self):
        if self.legend is not None:
            return
        self.legend = getLegend(self.config.legend_position[0],
                                self.config.legend_position[1],
                                self.config.legend_position[0]+self.config.legend_size[0],
                                self.config.legend_position[1]+self.config.legend_size[1])
        for hidx,hist in enumerate(self.histos):
            histo_class = hist.ClassName()
            if 'TGraph' not in histo_class:
                self.legend.AddEntry(hist, self.labels[hidx], 'l')
            else:
                self.legend.AddEntry(hist, self.labels[hidx], 'P')

        return


    def draw(self,
             text,
             options='',
             norm=False,
             y_log=False,
             x_log=False,
             y_min=None,
             y_max=None,
             x_min=None,
             x_max=None,
             y_axis_label=None,
             x_axis_label=None,
             v_lines=None,
             h_lines=None,
             do_profile=False):

        global p_idx
        global stuff

        self.formatHistos()
        self.createCanvas()
        if self.overlay:
            self.createLegend()

        self.canvas.cd()

        drawn_histos = []
        for hidx, hist in enumerate(self.histos):
            histo_class = hist.ClassName()

            opt = options
            if 'TGraph' in histo_class:
                opt = 'P'+options
            if hidx:
                opt = 'same,'+opt
            else:
                if 'TGraph' in histo_class:
                    opt = opt+'A'

            if not self.overlay:
                self.canvas.cd(hidx+1)
                if text:
                    newtext = '{}: {}'.format(self.labels[hidx], text)
                    rtext = getText(newtext, 0.15, 0.85)
                    rtext.Draw('same')
                    self.drawAdditionalText()

            d_hist = hist
            if norm:
                d_hist = hist.DrawNormalized(opt, 1.)
            else:
                d_hist.Draw(opt)

            if do_profile:
                profname = d_hist.GetName()+'_prof_'+str(p_idx)
                p_idx += 1
                prof = d_hist.ProfileX(d_hist.GetName()+'_prof_'+str(p_idx),
                                       1, -1, 's')
                prof.SetMarkerColor(2)
                prof.SetLineColor(2)
                prof.Draw('same')


            drawn_histos.append(d_hist)

            # we now set the axis properties
            y_min_value = y_min
            y_max_value = y_max

            if y_min is None:
                y_min_value = min([hist.GetBinContent(hist.GetMinimumBin()) for hist in drawn_histos if 'TGraph' not in hist.ClassName()] +
                                  [min(hist.GetY()) for hist in drawn_histos if 'TGraph' in hist.ClassName()])
            if y_max is None:
                y_max_value = max([hist.GetBinContent(hist.GetMaximumBin()) for hist in drawn_histos if 'TGraph' not in hist.ClassName()] +
                                  [max(hist.GetY()) for hist in drawn_histos if 'TGraph' in hist.ClassName()])*1.2

            for hist in drawn_histos:
                hist.GetXaxis().SetTitleOffset(1.4)
                hist.GetYaxis().SetRangeUser(y_min_value, y_max_value)
                if y_axis_label:
                    hist.GetYaxis().SetTitle(y_axis_label)
                if x_axis_label:
                    hist.GetXaxis().SetTitle(x_axis_label)
                if x_min is not None and x_max is not None:
                    if 'TGraph' not in hist.ClassName():
                        hist.GetXaxis().SetRangeUser(x_min, x_max)
                    else:
                        hist.GetXaxis().SetLimits(x_min, x_max)


        if self.legend is not None and len(self.histos) > 1:
            self.legend.Draw("same")

        if self.overlay:
            if text:
                rtext = getText(text, 0.15, 0.85)
                rtext.Draw("same")
            self.drawAdditionalText()

        pad_range = range(0, 1)
        if not self.overlay:
            pad_range = range(1, len(self.histos)+1)

        self.canvas.Update()
        for pad_id in pad_range:
            self.canvas.cd(pad_id)
            if v_lines:
                for v_line_x in v_lines:
                    aline = ROOT.TLine(v_line_x, ROOT.gPad.GetUymin(), v_line_x,  ROOT.gPad.GetUymax())
                    aline.SetLineStyle(2)
                    aline.Draw("same")
                    stuff.append(aline)
            if h_lines:
                for h_line_y in h_lines:
                    aline = ROOT.TLine(ROOT.gPad.GetUxmin(), h_line_y, ROOT.gPad.GetUxmax(),  h_line_y)
                    aline.SetLineStyle(2)
                    aline.Draw("same")
                    stuff.append(aline)
            if y_log:
                ROOT.gPad.SetLogy()
            if x_log:
                ROOT.gPad.SetLogx()

            ROOT.gPad.Update()

        self.canvas.Draw()
        return

    def write(self, name, ext='pdf'):
        self.canvas.SaveAs('{}.{}'.format(name, ext))
        return


def draw(histograms,
         labels,
         options='',
         norm=False,
         logy=False,
         min_y=None,
         max_y=None,
         min_x=None,
         max_x=None,
         text=None,
         y_axis_label=None,
         x_axis_label=None,
         v_lines=None,
         h_lines=None,
         do_stats=False,
         do_profile=False,
         do_ratio=False,
         do_write=False,
         write_name=None,
         legend_position=None):
    ROOT.gStyle.SetOptStat(False)

    config = rleg_config
    if do_write:
        config = tdr_config
    if do_stats:
        config.do_stats = do_stats
    if legend_position is not None:
        config.legend_position = (legend_position[0],
                                  legend_position[1])
        config.legend_size = (legend_position[2]-legend_position[0],
                              legend_position[3]-legend_position[1])

    dm = DrawMachine(config)
    dm.addHistos(histograms, labels)
    dm.draw(text=text,
            options=options,
            norm=norm,
            y_log=logy,
            y_min=min_y,
            y_max=max_y,
            x_min=min_x,
            x_max=max_x,
            y_axis_label=y_axis_label,
            x_axis_label=x_axis_label,
            v_lines=v_lines,
            h_lines=h_lines,
            do_profile=False,
            # do_ratio=False,
           )
    if do_write:
        dm.write(name=write_name)
    return dm


files = {}
file_keys = {}


class RootFile:
    def __init__(self, file_name):
        global file
        self.file_name = file_name
        if self.file_name not in files.keys():
            print 'get file: {}'.format(self.file_name)
            files[self.file_name] = ROOT.TFile(self.file_name)
        self._file = files[self.file_name]
        self._file_keys = None

    def cd(self):
        self._file.cd()

    def GetListOfKeys(self):
        global file_keys
        if self.file_name not in file_keys.keys():
            print 'get list'
            file_keys[self.file_name] = self._file.GetListOfKeys()
        self._file_keys = file_keys[self.file_name]
        return self._file_keys


class Sample():
    def __init__(self, name, label, version=None, type=None):
        self.name = name
        self.label = label
        if version:
            version = '_'+version
        else:
            version = ''
        self.histo_filename = '../plots1/histos_{}{}.root'.format(self.name, version)
        self.histo_file = None
        self.type = type
        self.oldStyle = False

    def __repr__(self):
        return '<{} {}, {}>'.format(self.__class__.__name__, self.histo_filename, self.type)

    def open_file(self):
        if self.histo_file is None:
            self.histo_file = ROOT.TFile(self.histo_filename, 'r')

    def build_file_primitive_index_oldStyle(self):
        # FIXME: this is really hugly
        composite_classes = {('GenParticleHistos', 'h_effNum_'): 'HistoSetEff',
                             ('GenParticleHistos', 'h_effDen_'): 'HistoSetEff'}

        self.open_file()
        self.histo_file.cd()
        data_list = []
        classtype = 'CalibrationHistos'
        print '--- {}'.format(classtype)
        print '# of plots: {}'.format(len(self.histo_file.GetListOfKeys()))
        # same primitives (tp, tp_sel, gen_sel) applies to several entries
        key_set = set()
        for histo in self.histo_file.GetListOfKeys():
            # print histo.GetName()
            name_parts = histo.GetName().split('_')
            cltype, tp, tp_sel, gen_sel = None, None, None, None
            if len(name_parts) == 4:
                cltype, tp, tp_sel, gen_sel = classtype, name_parts[0], name_parts[1], name_parts[2]
            else:
                # this is a histo in a HistoSet class.. we need to handle this differently
                composite_class = composite_classes[(classtype, '{}_{}_'.format(name_parts[0], name_parts[1]))]
                cltype, tp, tp_sel, gen_sel = composite_class, name_parts[2], name_parts[3], name_parts[4]
            key_set.add((cltype, tp, tp_sel, gen_sel))
        print '# of primitives: {}'.format(len(key_set))
        for cltype, tp, tp_sel, gen_sel in key_set:
            data_list.append({'classtype': cltype,
                              'tp': tp,
                              'tp_sel': tp_sel,
                              'gen_sel': gen_sel})

        return pd.DataFrame(data_list)

    def build_file_primitive_index(self):
        if self.oldStyle:
            return self.build_file_primitive_index_oldStyle()
        # FIXME: this is really hugly
        composite_classes = {('GenParticleHistos', 'h_effNum_'): 'HistoSetEff',
                             ('GenParticleHistos', 'h_effDen_'): 'HistoSetEff'}

        self.open_file()
        self.histo_file.cd()
        data_list = []
        for key in self.histo_file.GetListOfKeys():
            # first level is classtype
            classtype = key.GetName()
            print '--- {}'.format(classtype)
            file_dir = self.histo_file.GetDirectory(key.GetName())
            print '# of plots: {}'.format(len(file_dir.GetListOfKeys()))
            # same primitives (tp, tp_sel, gen_sel) applies to several entries
            key_set = set()
            for histo in file_dir.GetListOfKeys():
                # print histo.GetName()
                name_parts = histo.GetName().split('_')
                cltype, tp, tp_sel, gen_sel = None, None, None, None
                if len(name_parts) == 3:
                    cltype, tp, tp_sel, gen_sel = classtype, name_parts[0], name_parts[1], None
                elif len(name_parts) == 4:
                    cltype, tp, tp_sel, gen_sel = classtype, name_parts[0], name_parts[1], name_parts[2]
                else:
                    # this is a histo in a HistoSet class.. we need to handle this differently
                    composite_class = composite_classes[(classtype, '{}_{}_'.format(name_parts[0], name_parts[1]))]
                    cltype, tp, tp_sel, gen_sel = composite_class, name_parts[2], name_parts[3], name_parts[4]
                key_set.add((cltype, tp, tp_sel, gen_sel))
            print '# of primitives: {}'.format(len(key_set))
            for cltype, tp, tp_sel, gen_sel in key_set:
                data_list.append({'classtype': cltype,
                                  'tp': tp,
                                  'tp_sel': tp_sel,
                                  'gen_sel': gen_sel})

        return pd.DataFrame(data_list)

    def print_file_primitive_index(self):
        primitive_index = self.build_file_primitive_index()
        classtypes = primitive_index.classtype.unique()
        for classtype in classtypes:
            print '- HistoClass: {}'.format(classtype)
            tps = primitive_index[primitive_index.classtype == classtype].tp.unique()
            for tp in tps:
                print '  - TP: {}'.format(tp)
                tp_sels = primitive_index[(primitive_index.classtype == classtype) &
                                          (primitive_index.tp == tp)].tp_sel.unique()
                for tp_sel in tp_sels:
                    gen_sels = primitive_index[(primitive_index.classtype == classtype) &
                                               (primitive_index.tp == tp) &
                                               (primitive_index.tp_sel == tp_sel)].gen_sel.unique()
                    print '    - TP SEL: {} -> GEN SEL: {}'.format(tp_sel, gen_sels)


class HProxy:
    def __init__(self, classtype, tp, tp_sel, gen_sel, root_file):
        self.classtype = classtype
        self.tp = tp
        self.tp_sel = tp_sel
        self.gen_sel = gen_sel
        self.root_file = root_file
        self.instance = None

    def get(self, debug=False):
        if self.instance is None:
            name = '{}_{}_{}'.format(self.tp, self.tp_sel, self.gen_sel)
            if self.gen_sel is None:
                name = '{}_{}'.format(self.tp, self.tp_sel)
            if debug:
                print '-- HProxy:Get: {}'.format(name)
            self.instance = self.classtype(name, self.root_file, debug)
        return self.instance


class HPlot:
    def __init__(self, samples, labels_dict):
        self.samples_ = samples

        # populate the label dict
        self.labels_dict = labels_dict
        for sample in samples:
            self.labels_dict[sample.type] = sample.type
        self.labels_dict.update({'PU0': 'PU0', 'PU200': 'PU200'})

        self.data = pd.DataFrame(columns=['sample', 'pu', 'tp', 'tp_sel', 'gen_sel', 'classtype', 'histo'])


    def create_histo_proxies(self, classtype):
        for sample in self.samples_:
            histo_primtive_index = sample.build_file_primitive_index()
            class_primitive_index = histo_primtive_index[histo_primtive_index.classtype == str(classtype).split('.')[-1]]
            if class_primitive_index.empty:
                print '*** ERROR: No entry for class: {} in the primtive index!'.format(classtype)
                return

            for index, row in class_primitive_index.iterrows():
                self.data = self.data.append({'sample': sample.type,
                                              'pu': sample.label,
                                              'tp': row.tp,
                                              'tp_sel': row.tp_sel,
                                              'gen_sel': row.gen_sel,
                                              'classtype': classtype,
                                              'histo': HProxy(classtype,
                                                              row.tp,
                                                              row.tp_sel,
                                                              row.gen_sel,
                                                              sample.histo_file)},
                                             ignore_index=True)


    def get_histo(self,
                  classtype,
                  sample=None,
                  pu=None,
                  tp=None,
                  tp_sel=None,
                  gen_sel=None,
                  debug=False):
        histo = None
        labels = []
        text = ''

        query = '(pu == @pu) & (tp == @tp) & (tp_sel == @tp_sel) & (classtype == @classtype)'
        if gen_sel is not None:
            query += ' & (gen_sel == @gen_sel)'
        else:
            query += ' & (gen_sel.isnull())'
        if sample is not None:
            query += '& (sample == @sample)'

        histo_df = self.data.query(query)

        if histo_df.empty:
            print 'No match found for: pu: {}, tp: {}, tp_sel: {}, gen_sel: {}, classtype: {}'.format(pu, tp, tp_sel, gen_sel, classtype)
            return None, None, None
        if debug:
            print histo_df

        field_counts = histo_df.apply(lambda x: len(x.unique()))
        label_fields = []
        text_fields = []
        # print field_counts
        for field in field_counts.iteritems():
            if(field[1] > 1 and field[0] != 'histo'):
                label_fields.append(field[0])
            if(field[1] == 1 and field[0] != 'histo' and field[0] != 'classtype' and field[0] != 'sample'):
                if(gen_sel is None and field[0] == 'gen_sel'):
                    continue
                text_fields.append(field[0])
#         print 'label fields: {}'.format(label_fields)
#         print 'text fields: {}'.format(text_fields)

        for item in histo_df[label_fields].iterrows():
            labels.append(', '.join([self.labels_dict[tx] for tx in item[1].values if self.labels_dict[tx] != '']))

        # print labels
        text = ', '.join([self.labels_dict[fl] for fl in histo_df[text_fields].iloc[0].values if self.labels_dict[fl] != ''])
        histo = [his.get(debug) for his in histo_df['histo'].values]
        return histo, labels, text


In [15]:
# %load samples.py
import ROOT
import pandas as pd

version = 'v101'
version_v8 = 'v108'

version_v9 = 'v122'

# -------------------------------------------------------------------------

samples_ele = [
    Sample('ele_flat2to100_PU0', 'PU0', version, 'ele'),
    Sample('ele_flat2to100_PU200', 'PU200', version, 'ele')
    ]

samples_ele_V8 = [
    Sample('ele_flat2to100_PU0', 'PU0', version, 'V8'),
    Sample('ele_flat2to100_PU200', 'PU200', version, 'V8')
    ]


samples_ele_V9 = [
    Sample('all_flat5to80_PU0', 'PU0', 'v75t', 'V9'),
    Sample('all_flat5to80_PU200', 'PU200', 'v75t', 'V9')
    ]

samples_photons = [
    Sample('photon_flat8to150_PU0', 'PU0', version, 'photon'),
    Sample('photon_flat8to150_PU200', 'PU200', version, 'photon')
    ]

samples_pions = [
    Sample('pion_flat2to100_PU0', 'PU0', version, 'pions'),
    ]

# samples_nugus = [
#     Sample('nugun_alleta_pu0', 'PU0', version, 'mb'),
#     Sample('nugun_alleta_pu200', 'PU200', version, 'mb')
#     ]


samples_nugunrates = [
    Sample('nugun_alleta_pu200_rate', 'PU200', version, 'new-clusters')
    ]


samples_nugunrates_V8 = [
    Sample('nugun_alleta_pu200_rate', 'PU200', version_v8, 'hybrid')
    ]

samples_nugunrates_V9 = [
    Sample('nugun_alleta_pu200_rate', 'PU200', version_v9, 'new-ID')
    ]








In [36]:
# %load python/settings.py

# === samples =====================================================
import pprint
import python.plotters_config as plotters



samples = []

# samples += samples_nugunrates
# samples += samples_nugunrates_V8
samples += samples_nugunrates_V9

for smp in samples:
    smp.open_file()


sample = 'new-ID'

# === TP ==========================================================
tps = ['HMvDR', 
       'EG',
       'EGBRL',
       'EGALL',
       'TkEleEL',
       'TkEleELBRL',
       'TkEleELALL',
      ]

# === Load the Histo Primitives ====================================
histo_primitives = samples[0].build_file_primitive_index()


# print histo_primitives.data.unique()
# === TP selections ================================================
tp_select = {}

for tp in tps:
    tp_select[tp] = histo_primitives[histo_primitives.tp == tp].tp_sel.unique().tolist()

# ==== GEN selections ===============================================
gen_select ={}
for tp in tps:
    gen_select[tp] = histo_primitives[histo_primitives.tp == tp].gen_sel.unique().tolist()


import pprint
pp = pprint.PrettyPrinter(indent=4)
print '--- TPs: '
pp.pprint(tps)
print '--- TP selections:'
pp.pprint(tp_select)
print '--- GEN selections:'
pp.pprint(gen_select)


--- RateHistos
# of plots: 432
# of primitives: 144
--- TPs: 
['HMvDR', 'EG', 'EGBRL', 'EGALL', 'TkEleEL', 'TkEleELBRL', 'TkEleELALL']
--- TP selections:
{   'EG': [   'EGq5EtaDE',
              'EGq2EtaD',
              'EGq4EtaBCDE',
              'EGq5EtaBC',
              'EGq3EtaBC',
              'EGq3EtaBCD',
              'EGq2',
              'EGq2EtaBCDE',
              'EGq4EtaBC',
              'EGq5EtaD',
              'EGq2EtaBCD',
              'EGq4',
              'EGq5EtaBCD',
              'EGq3',
              'EGq2EtaDE',
              'EGq4EtaD',
              'EGq4EtaDE',
              'EGq3EtaD',
              'EGq4EtaBCD',
              'EGq5',
              'EGq2EtaBC',
              'EGq5EtaBCDE',
              'EGq3EtaDE',
              'EGq3EtaBCDE'],
    'EGALL': ['EtaFLooseTkID', 'EtaF', 'LooseTkID', 'all', 'EtaA', 'EtaBC'],
    'EGBRL': ['EtaF', 'EtaFLooseTkID', 'all', 'LooseTkID'],
    'HMvDR': [],
    'TkEleEL': [   'EGq4EtaBC',
                   'EGq

In [37]:
ROOT.gStyle.SetPadGridX(True)
ROOT.gStyle.SetPadGridY(True)


In [18]:
import python.collections as collections
import python.selections as selections

labels_dict = {}

evm = collections.EventManager()
labels_dict.update(evm.get_labels())
selm = selections.SelectionManager()
labels_dict.update(selm.get_labels())



In [19]:
for tp in tps:
    print 'TP: {}'.format(tp)
    print histo_primitives[histo_primitives.tp == tp].tp_sel.unique()
    print histo_primitives[histo_primitives.tp == tp].gen_sel.unique()

TP: HMvDR
[]
[]
TP: EG
['EGq5EtaDE' 'EGq2EtaD' 'EGq4EtaBCDE' 'EGq5EtaBC' 'EGq3EtaBC' 'EGq3EtaBCD'
 'EGq2' 'EGq2EtaBCDE' 'EGq4EtaBC' 'EGq5EtaD' 'EGq2EtaBCD' 'EGq4'
 'EGq5EtaBCD' 'EGq3' 'EGq2EtaDE' 'EGq4EtaD' 'EGq4EtaDE' 'EGq3EtaD'
 'EGq4EtaBCD' 'EGq5' 'EGq2EtaBC' 'EGq5EtaBCDE' 'EGq3EtaDE' 'EGq3EtaBCDE']
[None]
TP: EGBRL
['EtaF' 'EtaFLooseTkID' 'all' 'LooseTkID']
[None]
TP: EGALL
['EtaFLooseTkID' 'EtaF' 'LooseTkID' 'all' 'EtaA' 'EtaBC']
[None]
TP: TkEleEL
['EGq4EtaBC' 'EGq4Iso0p2EtaD' 'EGq2EtaDE' 'EGq4Iso0p2EtaBC' 'EGq3EtaDE'
 'EGq3EtaBCDE' 'EGq3Iso0p3EtaBCD' 'EGq3Iso0p2EtaBC' 'EGq4' 'EGq2EtaBCD'
 'EGq5' 'EGq3Iso0p2' 'EGq5EtaD' 'EGq4Iso0p3EtaBC' 'EGq2EtaBCDE'
 'EGq2Iso0p3EtaBCD' 'EGq2Iso0p2EtaBCDE' 'EGq2Iso0p1EtaBCDE' 'EGq3EtaBC'
 'EGq3Iso0p1EtaBC' 'EGq3Iso0p1EtaD' 'EGq4Iso0p2EtaBCDE' 'EGq3EtaBCD'
 'EGq4Iso0p1EtaD' 'EGq5EtaDE' 'EGq5Iso0p3EtaDE' 'EGq2Iso0p2EtaBC'
 'EGq5Iso0p1' 'EGq2Iso0p1' 'EGq2Iso0p3EtaD' 'EGq5Iso0p2' 'EGq5Iso0p3EtaBC'
 'EGq4Iso0p2EtaBCD' 'EGq3Iso0p1EtaBCDE' 'EGq3Iso0p3E

In [20]:
histo_primitives[:10]

,classtype,gen_sel,tp,tp_sel
0,RateHistos,None,TkEleEL,EGq4EtaBC
1,RateHistos,None,TkEleELBRL,EtaFIso0p1
2,RateHistos,None,TkEleEL,EGq4Iso0p2EtaD
3,RateHistos,None,TkEleEL,EGq2EtaDE
4,RateHistos,None,TkEleEL,EGq4Iso0p2EtaBC
5,RateHistos,None,TkEleELBRL,all
6,RateHistos,None,TkEleEL,EGq3EtaDE
7,RateHistos,None,TkEleELBRL,LooseTkID
8,RateHistos,None,TkEleEL,EGq3EtaBCDE
9,RateHistos,None,EGBRL,EtaF


In [21]:
%%time

import python.l1THistos as histos

hplot = HPlot(samples, labels_dict)

hplot.create_histo_proxies(histos.RateHistos)


--- RateHistos
# of plots: 432
# of primitives: 144
CPU times: user 595 ms, sys: 59.3 ms, total: 654 ms
Wall time: 614 ms


In [11]:
%%time
hplot = HPlot(samples, tp_labels, tp_selection_labels, [])

for tp in tps:
    print tp
    print histo_primitives[histo_primitives.data == tp].data_sel.unique()
    print samples
    hplot.cache_histo(classtype=histos.RateHistos, 
                      samples=samples,
                      pus=[],
                      tps=[tp],
                      tp_sels=histo_primitives[histo_primitives.data == tp].data_sel.unique(),
                      gen_sels=None)


HMvDR
['all' 'EtaD' 'EtaDE' 'EtaBC' 'EtaBCD' 'EtaBCDE' 'Em' 'EmEtaD' 'EmEtaDE'
 'EmEtaBC' 'EmEtaBCD' 'EmEtaBCDE']
[<Sample ../plots1/histos_nugun_alleta_pu200_rate_v108.root, hybrid>, <Sample ../plots1/histos_nugun_alleta_pu200_rate_v122.root, new-ID>]
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v108.root, hybrid>
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v108.root, hybrid> HMvDR all None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v108.root, hybrid> HMvDR EtaD None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v108.root, hybrid> HMvDR EtaDE None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v108.root, hybrid> HMvDR EtaBC None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v108.root, hybrid> HMvDR EtaBCD None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v108.root, hybrid> HMvDR EtaBCDE None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v108.root, hybrid> HMvDR Em None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v108.root, hybrid> HMvDR EmEtaD None
<

<Sample ../plots1/histos_nugun_alleta_pu200_rate_v108.root, hybrid> EGALL EtaF None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v108.root, hybrid> EGALL EtaA None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v108.root, hybrid> EGALL EtaBC None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v108.root, hybrid> EGALL LooseTkID None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v108.root, hybrid> EGALL EtaFLooseTkID None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v122.root, new-ID>
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v122.root, new-ID> EGALL all None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v122.root, new-ID> EGALL EtaF None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v122.root, new-ID> EGALL EtaA None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v122.root, new-ID> EGALL EtaBC None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v122.root, new-ID> EGALL LooseTkID None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v122.root, new-ID> EGAL

<Sample ../plots1/histos_nugun_alleta_pu200_rate_v108.root, hybrid> TkEleEL EGq4Iso0p1 None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v108.root, hybrid> TkEleEL EGq4Iso0p1EtaD None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v108.root, hybrid> TkEleEL EGq4Iso0p1EtaDE None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v108.root, hybrid> TkEleEL EGq4Iso0p1EtaBC None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v108.root, hybrid> TkEleEL EGq4Iso0p1EtaBCD None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v108.root, hybrid> TkEleEL EGq4Iso0p1EtaBCDE None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v108.root, hybrid> TkEleEL EGq4Iso0p3 None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v108.root, hybrid> TkEleEL EGq4Iso0p3EtaD None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v108.root, hybrid> TkEleEL EGq4Iso0p3EtaDE None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v108.root, hybrid> TkEleEL EGq4Iso0p3EtaBC None
<Sample ../plots1/histos_nugun_alleta_p

<Sample ../plots1/histos_nugun_alleta_pu200_rate_v122.root, new-ID> TkEleEL EGq4EtaBCD None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v122.root, new-ID> TkEleEL EGq4EtaBCDE None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v122.root, new-ID> TkEleEL EGq4Iso0p2 None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v122.root, new-ID> TkEleEL EGq4Iso0p2EtaD None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v122.root, new-ID> TkEleEL EGq4Iso0p2EtaDE None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v122.root, new-ID> TkEleEL EGq4Iso0p2EtaBC None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v122.root, new-ID> TkEleEL EGq4Iso0p2EtaBCD None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v122.root, new-ID> TkEleEL EGq4Iso0p2EtaBCDE None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v122.root, new-ID> TkEleEL EGq4Iso0p1 None
<Sample ../plots1/histos_nugun_alleta_pu200_rate_v122.root, new-ID> TkEleEL EGq4Iso0p1EtaD None
<Sample ../plots1/histos_nugun_alleta_pu200_rate

In [32]:
hplot.data['sample'].unique()

array(['new-ID'], dtype=object)

In [75]:
hplot.data[:10]

,sample,pu,tp,tp_sel,gen_sel,classtype,histo
0,new-ID,PU200,TkEleEL,EGq4EtaBC,None,python.l1THistos.RateHistos,<__main__.HProxy instance at 0x11cbeb7a0>
1,new-ID,PU200,TkEleELBRL,EtaFIso0p1,None,python.l1THistos.RateHistos,<__main__.HProxy instance at 0x11cbe97e8>
2,new-ID,PU200,TkEleEL,EGq4Iso0p2EtaD,None,python.l1THistos.RateHistos,<__main__.HProxy instance at 0x11cbeaa70>
3,new-ID,PU200,TkEleEL,EGq2EtaDE,None,python.l1THistos.RateHistos,<__main__.HProxy instance at 0x1195e5f80>
4,new-ID,PU200,TkEleEL,EGq4Iso0p2EtaBC,None,python.l1THistos.RateHistos,<__main__.HProxy instance at 0x10dcf4d40>
5,new-ID,PU200,TkEleELBRL,all,None,python.l1THistos.RateHistos,<__main__.HProxy instance at 0x11cbebf38>
6,new-ID,PU200,TkEleEL,EGq3EtaDE,None,python.l1THistos.RateHistos,<__main__.HProxy instance at 0x10dd04c20>
7,new-ID,PU200,TkEleELBRL,LooseTkID,None,python.l1THistos.RateHistos,<__main__.HProxy instance at 0x11cbec1b8>
8,new-ID,PU200,TkEleEL,EGq3EtaBCDE,None,python.l1THistos.RateHistos,<__main__.HProxy instance at 0x11cbecc20>
9,new-ID,PU200,EGBRL,EtaF,None,python.l1THistos.RateHistos,<__main__.HProxy instance at 0x10cf933b0>


#### 3D clusters matched to GEN

In [76]:
# version_V9 = 'v113'
scaling_params =  pd.read_csv(r'scaling_params_{}.csv'.format(version_v9))

In [77]:
scaling_params

,Unnamed: 0,tp,tp_sel,params,pu
0,0,EG,EGq5EtaBC,"(5.795971470752347, 1.1107818293529574)",PU0
1,1,TkEleEL,EGq5EtaBC,"(5.020946648772676, 1.1318219292495295)",PU0
2,2,EG,EGq5EtaBC,"(7.980993336197056, 1.0360841147270883)",PU200
3,3,TkEleEL,EGq5EtaBC,"(5.390144476979888, 1.1158208031274417)",PU200


In [78]:
eval(scaling_params[(scaling_params.pu == 'PU200') & (scaling_params.tp == 'EG') & (scaling_params.tp_sel == 'EGq3EtaBC')].params.iloc[0])

IndexError: single positional indexer is out-of-bounds

In [39]:
def convertRateToGraph(histo_rate, params=None):
    xaxis = histo_rate.GetXaxis()
    fNpoints = xaxis.GetNbins()
    fX = [0]*fNpoints
    fEX = [0]*fNpoints
    fY = [0]*fNpoints
    fEY = [0]*fNpoints

    graph = ROOT.TGraphAsymmErrors()
    graph.Set(fNpoints)
    for i in range(0, fNpoints):
        if params is None:
            fX[i] = xaxis.GetBinLowEdge(i + 1)
        else:
            fX[i] = params[0] + params[1]*xaxis.GetBinLowEdge(i + 1)
        fY[i] = histo_rate.GetBinContent(i + 1)
        fEX[i] = histo_rate.GetBinWidth(i + 1) * ROOT.gStyle.GetErrorX()
        fEY[i] = histo_rate.GetBinError(i + 1)
    return fX, fY, fEX, fEX, fEY, fEY


In [40]:
for tp in ['EG']:
    for tp_sel in ['EGq5EtaBC']:

        hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', tp, tp_sel, None)
        for hist in hsets:
            hist.normalize(31000)
        draw([his.h_pt_graph('online', 'rate [kHz]', convertRateToGraph) for his in hsets], labels, text=text,  min_y =1, max_y = 40000 ,logy=True)
        draw([his.h_pt_graph('offline', 'rate [kHz]', lambda x: convertRateToGraph(x, eval(scaling_params[(scaling_params.pu == 'PU200') & 
                                                                                                  (scaling_params.tp == tp) & 
                                                                                                  (scaling_params.tp_sel == tp_sel)].params.iloc[0]))) for his in hsets], 
             labels, text=text,  min_y =1, max_y = 40000 ,logy=True)


normalize to 31000


In [41]:
for tp in ['HMvDRshapeDrCalib']:
    for tp_sel in ['EmEtaBC']:

        hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', tp, tp_sel, None)
        for hist in hsets:
            hist.normalize(31000)
        draw([his.h_pt_graph('online', 'rate [kHz]', convertRateToGraph) for his in hsets], labels, text=text,  min_y =1, max_y = 40000 ,logy=True)
        draw([his.h_pt_graph('offline', 'rate [kHz]', lambda x: convertRateToGraph(x, eval(scaling_params[(scaling_params.pu == 'PU200') & 
                                                                                                  (scaling_params.tp == tp) & 
                                                                                                  (scaling_params.tp_sel == tp_sel)].params.iloc[0]))) for his in hsets], 
             labels, text=text,  min_y =1, max_y = 40000 ,logy=True)


No match found for: pu: PU200, tp: HMvDRshapeDrCalib, tp_sel: EmEtaBC, gen_sel: None, classtype: python.l1THistos.RateHistos


TypeError: 'NoneType' object is not iterable

In [42]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', 'EG', ['EGq4EtaBC'], None)
for hist in hsets:
    hist.normalize(31000)
newset = [hsets[0].g_pt_online, hsets[0].g_pt_offline]
newlabels = ['no-calib online', 'no-calib offline']

hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', 'HMvDRshapeDrCalib', ['EmEtaBC'], None)
for hist in hsets:
    hist.normalize(31000)
newset.extend([hsets[0].g_pt_online, hsets[0].g_pt_offline])
newlabels.extend(['calib online', 'calib offline'])


draw(newset, newlabels, text=text,  min_y =1, max_y = 40000 ,logy=True, x_axis_label='p_{T} thresh. [GeV]')

normalize to 31000


AttributeError: RateHistos instance has no attribute 'g_pt_online'

In [43]:
sample

'new-ID'

In [79]:
for tp in ['EG', 'TkEleEL']:
    for tp_sel in ['EGq5EtaBC']:

        hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', tp, tp_sel, None)

        draw([his.h_pt_graph('online', 'rate [kHz]', convertRateToGraph) for his in hsets], labels, text=text,  min_y =1, max_y = 40000 ,logy=True)
        draw([his.h_pt_graph('offline', 'rate [kHz]', lambda x: convertRateToGraph(x, eval(scaling_params[(scaling_params.pu == 'PU200') & 
                                                                                                  (scaling_params.tp == tp) & 
                                                                                                  (scaling_params.tp_sel == tp_sel)].params.iloc[0]))) for his in hsets], 
             labels, text=text,  min_y =1, max_y = 40000 ,logy=True)








In [80]:
for tp in ['EG', 'TkEleEL']:
    for tp_sel in ['EGq5']:
        for hist in hsets:
            hist.normalize(31000)
        hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', tp, tp_sel, None)

        draw([his.h_pt_graph('online', 'rate [kHz]', convertRateToGraph) for his in hsets], labels, text=text,  min_y =1, max_y = 40000 ,logy=True)
        draw([his.h_pt_graph('offline', 'rate [kHz]', lambda x: convertRateToGraph(x, eval(scaling_params[(scaling_params.pu == 'PU200') & 
                                                                                                  (scaling_params.tp == tp) & 
                                                                                                  (scaling_params.tp_sel == 'EGq5EtaBC')].params.iloc[0]))) for his in hsets], 
             labels, text=text,  min_y =1, max_y = 40000 ,logy=True)



normalize to 31000


In [ ]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', 'EG', ['EGq3EtaBC', 'EGq4EtaBC', 'EGq5EtaBC'], None)

draw([his.g_pt_offline for his in hsets], labels, text=text,  min_y =1, max_y = 40000 ,logy=True, x_axis_label='offline p_{T} thresh. [GeV]')

In [81]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', 'EG', ['EGq5EtaBC'], None)
for hist in hsets:
    hist.normalize(31000)
newset = [hsets[0].g_pt_online, hsets[0].g_pt_offline]
newlabels = ['EG online', 'EG offline']


draw([his.g_pt_online for his in hsets], labels, text=text,  min_y =1, max_y = 40000 ,logy=True, x_axis_label='online p_{T} thresh. [GeV]')
draw([his.g_pt_offline for his in hsets], labels, text=text,  min_y =1, max_y = 40000 ,logy=True, x_axis_label='offline p_{T} thresh. [GeV]')

hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', 'TkEleEL', ['EGq5EtaBC'], None)
for hist in hsets:
    hist.normalize(31000)
draw([his.g_pt_online for his in hsets], labels, text=text,  min_y =1, max_y = 40000 ,logy=True, x_axis_label='online p_{T} thresh. [GeV]')
draw([his.g_pt_offline for his in hsets], labels, text=text,  min_y =1, max_y = 40000 ,logy=True, x_axis_label='offline p_{T} thresh. [GeV]')

newset.append(hsets[0].g_pt_online)
newlabels.append('TkEle online')
newset.append(hsets[0].g_pt_offline)
newlabels.append('TkEle offline')

draw(newset, newlabels, text=text,  min_y =1, max_y = 40000 ,logy=True, x_axis_label='p_{T} thresh. [GeV]')

In [82]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', 'EG', ['EGq5EtaBCD', 'EGq5'], None)
for hist in hsets:
    hist.normalize(31000)
draw([his.h_pt for his in hsets], labels, text=text,  min_y =1, max_y = 40000 ,logy=True, x_axis_label='online p_{T} thresh. [GeV]')

normalize to 31000


In [83]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', 'TkEleEL', ['EGq2EtaBC','EGq3EtaBC', 'EGq4EtaBC', 'EGq5EtaBC'], None)
draw([his.g_pt_offline for his in hsets], labels, text=text,  min_y =1, max_y = 40000 ,logy=True, x_axis_label='offline p_{T} thresh. [GeV]')

AttributeError: RateHistos instance has no attribute 'g_pt_offline'

In [84]:
for tp in ['EG', 'TkEleEL']:
    for tp_sel in ['EGq3EtaBC', 'EGq5EtaBC']:
        hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', tp, tp_sel, None)
        for hist in hsets:
            hist.normalize(31000)
        # drawSeveral([his.hreso.h_ptResVpt for his in hsets], labels,  'COLZ', do_profile=True, text=text)
        draw([his.h_pt for his in hsets], labels, text=text, logy=True, h_lines=[100], v_lines=[45])


Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale


In [85]:
# 'HMvDRCalib','HMvDRshapeDrCalib',
hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['HMvDR', 'HMvDREmInt' ], 'EmEtaBC', None)
for hist in hsets:
    hist.normalize(31000)
draw([his.h_pt for his in hsets], labels, text=text, logy=True)

No match found for: pu: PU200, tp: ['HMvDR', 'HMvDREmInt'], tp_sel: EmEtaBC, gen_sel: None, classtype: python.l1THistos.RateHistos


TypeError: 'NoneType' object is not iterable

In [86]:
# 'HMvDRCalib','HMvDRshapeDrCalib',
hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['HMvDR', 'HMvDRCalib', 'HMvDREmInt' ], 'EmEtaBC', None)
for hist in hsets:
    hist.normalize(31000)
draw([his.h_pt for his in hsets], labels, text=text, logy=True)

No match found for: pu: PU200, tp: ['HMvDR', 'HMvDRCalib', 'HMvDREmInt'], tp_sel: EmEtaBC, gen_sel: None, classtype: python.l1THistos.RateHistos


TypeError: 'NoneType' object is not iterable

In [ ]:
sample

In [87]:
# 'HMvDRCalib','HMvDRshapeDrCalib',
hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['EG', 'TkEle', 'TkEleEL'], ['EGq3EtaBC', 'EGq4EtaBC', 'EGq5EtaBC'], None)
for hist in hsets:
    hist.normalize(31000)
draw([his.h_pt for his in hsets], labels, text=text, logy=True)

normalize to 31000


Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale


In [46]:
# 'HMvDRCalib','HMvDRshapeDrCalib',
hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['EG'], ['EGq3EtaBC', 'EGq4EtaBC', 'EGq5EtaBC'], None)

for hist in hsets:
    hist.normalize(31000)
draw([his.h_pt for his in hsets], labels, text=text, logy=True)

normalize to 31000


In [47]:
dm = DrawMachine(rleg_config)
hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['EG'], ['EGq3EtaBC', 'EGq4EtaBC', 'EGq5EtaBC'], None)
for hist in hsets:
    hist.normalize(31000)
dm.addHistos([his.h_pt for his in hsets], labels)
dm.draw(text=text, y_log=True)

In [48]:
sample

'new-ID'

In [49]:
#HERE
# 'HMvDRCalib','HMvDRshapeDrCalib',
hsets, labels, text = hplot.get_histo(histos.RateHistos, ['new-ID', 'old-ID', 'new-clusters'], 'PU200', ['EG'], ['EGq4EtaBC'], None)
for hist in hsets:
    hist.normalize(31000)
labels = ['TP head', 'L1 menu version', 'backported']
draw([his.h_pt for his in hsets], labels, text=text, logy=True)

In [50]:
# 'HMvDRCalib','HMvDRshapeDrCalib',
hsets, labels, text = hplot.get_histo(histos.RateHistos, ['new-ID'], 'PU200', ['EG'], ['EGq3EtaBC', 'EGq4EtaBC'], None)
for hist in hsets:
    hist.normalize(31000)
draw([his.h_pt for his in hsets], labels, text=text, logy=True)

In [51]:
# 'HMvDRCalib','HMvDRshapeDrCalib',
hsets, labels, text = hplot.get_histo(histos.RateHistos, ['new-ID'], 'PU200', ['EG', 'TkEle','TkEleEL'], ['EGq3EtaBC'], None)
for hist in hsets:
    hist.normalize(31000)
draw([his.h_pt for his in hsets], labels, text=text, logy=True)

normalize to 31000


In [52]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, ['hybrid', 'new-ID'], 'PU200', ['TkEleEL'], ['EGq5EtaBC'], None)
for hist in hsets:
    hist.normalize(31000)
draw([his.h_pt for his in hsets], labels, text=text, logy=True)

Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale


In [53]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, ['hybrid', 'new-ID'], 'PU200', ['TkEleELALL'], ['LooseTkID'], None)
for hist in hsets:
    hist.normalize(31000)
draw([his.h_pt for his in hsets], labels, text=text, logy=True)

normalize to 31000


Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale


In [54]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, ['hybrid', 'new-ID'], 'PU200', ['TkEleELALL'], ['EtaFLooseTkID'], None)
for hist in hsets:
    hist.normalize(31000)
draw([his.h_pt for his in hsets], labels, text=text, logy=True)

normalize to 31000


In [55]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, ['new-ID'], 'PU200', ['TkEleEL'], ['EGq3EtaBC', 'EGq3Iso0p2EtaBC', 'EGq3Iso0p1EtaBC'], None)
for hist in hsets:
    hist.normalize(31000)
draw([his.h_pt for his in hsets], labels, text=text, logy=True)

normalize to 31000
normalize to 31000


Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale


In [56]:
# 'HMvDRCalib','HMvDRshapeDrCalib',
hsets, labels, text = hplot.get_histo(histos.RateHistos, ['new-ID', 'old-ID'], 'PU200', ['EG'], ['EGq4EtaBC'], None)
for hist in hsets:
    hist.normalize(31000)
draw([his.h_pt for his in hsets], labels, text=text, logy=True)

Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale


In [57]:
# 'HMvDRCalib','HMvDRshapeDrCalib',
hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['TkEle'], ['EGq3EtaBC', 'EGq4EtaBC'], None)
for hist in hsets:
    hist.normalize(31000)
draw([his.h_pt for his in hsets], labels, text=text, logy=True)

No match found for: pu: PU200, tp: ['TkEle'], tp_sel: ['EGq3EtaBC', 'EGq4EtaBC'], gen_sel: None, classtype: python.l1THistos.RateHistos


TypeError: 'NoneType' object is not iterable

In [ ]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, 'new-ID', 'PU200', ['EG'], 'EGq5EtaD', None)

for hist in hsets:
    hist.normalize(31000)
[his.h_pt_graph('online', 'rate [kHz]', convertRateToGraph) for his in hsets]
draw([his.g_pt_online for his in hsets], labels, text=text, min_y=1., max_y=75000, logy=True)

In [23]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, 'new-ID', 'PU200', ['EG'], 'EGq5EtaD', None)
dc = DrawConfig()
dm = DrawMachine(dc)
for hist in hsets:
    hist.normalize(31000)
[his.h_pt_graph('online', 'rate [kHz]', convertRateToGraph) for his in hsets]

dm.addHistos([his.g_pt_online for his in hsets], labels)
dm.draw(text=text, y_min=1., y_max=75000, x_min=0, x_max=60., y_log=True)

draw([his.g_pt_online for his in hsets], labels, text=text, min_y=1., max_y=75000, logy=True)

normalize to 31000
rate above p_{T} thresh.;; rate [kHz]
;; rate [kHz]


<ROOT.TCanvas object ("c_70b7c0") at 0x7ff5b0e11750>

In [89]:
# 'HMvDRCalib','HMvDRshapeDrCalib', 
dm = DrawMachine(tdr_config)
hsets, labels, text = hplot.get_histo(histos.RateHistos, 'new-ID', 'PU200', ['HMvDR'], 'all', None, debug=False)

for hist in hsets:
    hist.normalize(31000)
[his.h_pt_graph('online', 'rate [kHz]', convertRateToGraph) for his in hsets]

dm.addHistos([his.g_pt_online for his in hsets], ['w/o EG ID'])

hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['EG'], ['EGq5EtaBCDE'], None)

for hist in hsets:
    hist.normalize(31000)
[his.h_pt_graph('online', 'rate [kHz]', convertRateToGraph) for his in hsets]

dm.addHistos([his.g_pt_online for his in hsets], ['w/ EG ID'])


dm.draw(text='EG endcap, 1.52 < |#eta^{L1}| #leq 3.0',
        y_log=True,
        x_min=0., x_max=60.,
        y_min=1., y_max=75000,
        x_axis_label='threshold p_{T}^{L1} [GeV]'
        )

dm.write(name='figs/egammEndcap_EGID_EtaBCDE_rate')
# # print text
# draw([his.g_pt_online for his in final_haset], labels_final, text='EG endcap, 1.52 < |#eta^{L1}| #leq 2.8', logy=True,
#      min_x=0., max_x=60., min_y=1., max_y=75000, x_axis_label='threshold p_{T}^{L1}[GeV]',
#      do_write=True, write_name='figs/egammEndcap_EGID_EtaBCD_rate')

No match found for: pu: PU200, tp: ['HMvDR'], tp_sel: all, gen_sel: None, classtype: python.l1THistos.RateHistos


TypeError: 'NoneType' object is not iterable

In [59]:
# 'HMvDRCalib','HMvDRshapeDrCalib', 
final_haset = []
labels_final = []
hsets, labels, text = hplot.get_histo(histos.RateHistos, 'hybrid', 'PU200', ['HMvDR'], 'EtaBCD', None)

for hist in hsets:
    hist.normalize(31000)
[his.h_pt_graph('online', 'rate [kHz]', convertRateToGraph) for his in hsets]
[his.g_pt_online.GetXaxis().SetLimits(0, 60) for his in final_haset]

final_haset.append(hsets[0]) 
labels_final.append('w/o EG ID')
# hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['HMvDRCalib'], 'EmEtaBCD', None)
# for hist in hsets:
#     hist.normalize(31000)
# final_haset.append(hsets[0]) 
# labels_final.append('EG ID clusters')

hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['EG'], 'EGq5EtaBCD', None)

for hist in hsets:
    hist.normalize(31000)
[his.h_pt_graph('online', 'rate [kHz]', convertRateToGraph) for his in hsets]
[his.g_pt_online.GetXaxis().SetLimits(0, 60) for his in final_haset]

final_haset.append(hsets[0]) 
labels_final.append('w/ EG ID')
# print text
dm = draw([his.g_pt_online for his in final_haset], labels_final, text='EG endcap, 1.52 < |#eta^{L1}| #leq 2.8', logy=True,
     min_x=0., max_x=60., min_y=1., max_y=75000, x_axis_label='threshold p_{T}^{L1}[GeV]',
     do_write=True, write_name='figs/egammEndcap_EGID_EtaBCD_rate')
# dm.config = tdr_config
# dm.draw(text)


No match found for: pu: PU200, tp: ['HMvDR'], tp_sel: EtaBCD, gen_sel: None, classtype: python.l1THistos.RateHistos


TypeError: 'NoneType' object is not iterable

In [53]:
dm.histos

[<ROOT.TGraphAsymmErrors object ("cdae58") at 0x7ff5b0af2870>,
 <ROOT.TGraphAsymmErrors object ("29b5f3") at 0x7ff5b0e28b10>]

In [60]:
# 'HMvDRCalib','HMvDRshapeDrCalib', 
final_haset = []
labels_final = []
# hsets, labels, text = hplot.get_histo(histos.RateHistos, 'old-ID', 'PU200', ['HMvDR'], 'EtaBCD', None)

# for hist in hsets:
#     hist.normalize(31000)
# [his.h_pt_graph('online', 'rate [kHz]', convertRateToGraph) for his in hsets]
# [his.g_pt_online.GetXaxis().SetLimits(0, 60) for his in final_haset]

# final_haset.append(hsets[0]) 
# labels_final.append('w/o EG ID')
# hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['HMvDRCalib'], 'EmEtaBCD', None)
# for hist in hsets:
#     hist.normalize(31000)
# final_haset.append(hsets[0]) 
# labels_final.append('EG ID clusters')

hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['EGALL', 'TkEleELALL'], ['LooseTkID', 'Iso0p1'], None)

for hist in hsets:
    hist.normalize(31000)
[his.h_pt_graph('online', 'rate [kHz]', convertRateToGraph) for his in hsets]
[his.g_pt_online.GetXaxis().SetLimits(0, 60) for his in hsets]
final_haset.append(hsets[0]) 
labels_final.append('calorimeter-only')
labels_final.append('track-matched')
labels_final.append('track-matched + charged iso.')




# print text
draw([his.g_pt_online for his in hsets], labels_final, text='EG barrel & endcap', logy=True,
     min_x=0., max_x=60., min_y=1., max_y=75000, x_axis_label='threshold p_{T}^{L1} [GeV]',
     legend_position=(0.4, 0.65, 0.66, 0.75),
     do_write=True, write_name='figs/egammBarrelEndcap_EG_rate')

normalize to 31000
normalize to 31000


Info in <TCanvas::Print>: pdf file figs/egammBarrelEndcap_EG_rate.pdf has been created


In [61]:
# 'HMvDRCalib','HMvDRshapeDrCalib', 
final_haset = []
labels_final = []
# hsets, labels, text = hplot.get_histo(histos.RateHistos, 'old-ID', 'PU200', ['HMvDR'], 'EtaBCD', None)

# for hist in hsets:
#     hist.normalize(31000)
# [his.h_pt_graph('online', 'rate [kHz]', convertRateToGraph) for his in hsets]
# [his.g_pt_online.GetXaxis().SetLimits(0, 60) for his in final_haset]

# final_haset.append(hsets[0]) 
# labels_final.append('w/o EG ID')
# hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['HMvDRCalib'], 'EmEtaBCD', None)
# for hist in hsets:
#     hist.normalize(31000)
# final_haset.append(hsets[0]) 
# labels_final.append('EG ID clusters')

hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['EGALL', 'TkEleELALL'], ['EtaFLooseTkID', 'EtaFIso0p1'], None)

for hist in hsets:
    hist.normalize(31000)
[his.h_pt_graph('online', 'rate [kHz]', convertRateToGraph) for his in hsets]
[his.g_pt_online.GetXaxis().SetLimits(0, 60) for his in hsets]
final_haset.append(hsets[0]) 
labels_final.append('w/ EG ID')
# print text
draw([his.g_pt_online for his in hsets], labels, text=text+' barrel', logy=True,
     min_x=0., max_x=60., min_y=1., max_y=75000, x_axis_label='threshold p_{T}^{L1} [GeV]',
     do_write=True, write_name='egammBarrel_EG_rate')

normalize to 31000
normalize to 31000


Info in <TCanvas::Print>: pdf file egammBarrel_EG_rate.pdf has been created


In [90]:
# 'HMvDRCalib','HMvDRshapeDrCalib', 
final_haset = []
labels_final = []
# hsets, labels, text = hplot.get_histo(histos.RateHistos, 'old-ID', 'PU200', ['HMvDR'], 'EtaBCD', None)

# for hist in hsets:
#     hist.normalize(31000)
# [his.h_pt_graph('online', 'rate [kHz]', convertRateToGraph) for his in hsets]
# [his.g_pt_online.GetXaxis().SetLimits(0, 60) for his in final_haset]

# final_haset.append(hsets[0]) 
# labels_final.append('w/o EG ID')
# hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['HMvDRCalib'], 'EmEtaBCD', None)
# for hist in hsets:
#     hist.normalize(31000)
# final_haset.append(hsets[0]) 
# labels_final.append('EG ID clusters')

hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['EGALL', 'TkEleELALL'], ['EtaABCLooseTkID', 'EtaABCIso0p1'], None)

for hist in hsets:
    hist.normalize(31000)
[his.h_pt_graph('online', 'rate [kHz]', convertRateToGraph) for his in hsets]
[his.g_pt_online.GetXaxis().SetLimits(0, 60) for his in hsets]
final_haset.append(hsets[0]) 
labels_final.append('w/ EG ID')
# print text
draw([his.g_pt_online for his in hsets], labels, text=text+' endcap', logy=True,
     min_x=0., max_x=60., min_y=1., max_y=75000, x_axis_label='threshold p_{T}^{L1} [GeV]',
     do_write=True, write_name='egammEndcap_EG_rate')

No match found for: pu: PU200, tp: ['EGALL', 'TkEleELALL'], tp_sel: ['EtaABCLooseTkID', 'EtaABCIso0p1'], gen_sel: None, classtype: python.l1THistos.RateHistos


TypeError: 'NoneType' object is not iterable

In [63]:
final_haset = []
labels_final = []

hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['TkEleELALL'], 'EtaBC', None)
for hist in hsets:
    hist.normalize(31000)

final_haset.append(hsets[0])
labels_final.append('all')
hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['TkEleEL'], 'EGq5EtaBC', None)
for hist in hsets:
    hist.normalize(31000)

final_haset.append(hsets[0])
labels_final.append('endcap')


draw([his.h_pt for his in final_haset], labels_final, text=text, logy=True)
# for hist in hsets:
#     hist.normalize(31000)
# [his.h_pt_graph('online', 'rate [kHz]', convertRateToGraph) for his in hsets]
# [his.g_pt_online.GetXaxis().SetLimits(0, 60) for his in final_haset]

# final_haset.append(hsets[0]) 
# labels_final.append('w/ EG ID')
# # print text
# draw([his.g_pt_online for his in final_haset], labels_final, text='<PU> = 200, 1.52 < |#eta^{L1}| <= 2.8', logy=True,
#      min_x=0., max_x=60., min_y=1., max_y=75000, x_axis_label='threshold p_{T}^{L1}[GeV]',
#      do_write=True, write_name='egammEndcap_EGID_EtaBCD_rate')

normalize to 31000


Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale


In [64]:
final_haset = []
labels_final = []

hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['TkEleELALL'], 'EtaF', None)
for hist in hsets:
    hist.normalize(31000)

final_haset.append(hsets[0])
labels_final.append('all')
hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['TkEleELBRL'], 'EtaF', None)
for hist in hsets:
    hist.normalize(31000)

final_haset.append(hsets[0])
labels_final.append('barrel')


draw([his.h_pt for his in final_haset], labels_final, text=text, logy=True)

normalize to 31000
normalize to 31000


Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale
Error in <THistPainter::PaintInit>: Cannot set Y axis to log scale


In [65]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['TkEleELALL'], 'all', None)
for hist in hsets:
    hist.normalize(31000)
draw([his.h_pt for his in hsets], labels, text=text, logy=True,
     min_x=0., max_x=60., min_y=1., max_y=75000, x_axis_label='threshold p_{T}^{L1}[GeV]')


normalize to 31000


In [66]:
final_haset = []
labels_final = []

hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['TkEleELALL'], 'all', None)
for hist in hsets:
    hist.normalize(31000)

[his.h_pt_graph('online', 'rate [kHz]', convertRateToGraph) for his in hsets]
final_haset.append(hsets[0])

[his.g_pt_online.GetXaxis().SetLimits(0, 60) for his in final_haset]

labels_final.append('track-matched')

text_forprint='<PU> = 200'
draw([his.g_pt_online for his in final_haset], labels_final, text=text_forprint, logy=True,
     min_x=0., max_x=60., min_y=1., max_y=75000, x_axis_label='threshold p_{T}^{L1}[GeV]',
     do_write=True, write_name='egamma_TkEle_all_rate'
    )


Info in <TCanvas::Print>: pdf file egamma_TkEle_all_rate.pdf has been created


In [67]:
# final_haset = []
labels_final = []

hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['EG', 'TkEleEL'], 'EGq5EtaBC', None)
for hist in hsets:
    hist.normalize(31000)

[his.h_pt_graph('online', 'rate [kHz]', convertRateToGraph) for his in hsets]
# final_haset.append(hsets[0])
# final_haset.append(hsets[0])

[his.g_pt_online.GetXaxis().SetLimits(0, 60) for his in hsets]
labels_final.append('calorimeter only')
labels_final.append('track-matched')

text_forprint='EG endcap, 1.52 < |#eta^{L1}| #leq 2.4'
draw([his.g_pt_online for his in hsets], labels_final, text=text_forprint, logy=True,
     min_x=0., max_x=60., min_y=1., max_y=75000, x_axis_label='threshold p_{T}^{L1} [GeV]',
     do_write=True, write_name='figs/egammEndcap_TkEle_EtaBC_rate'
    )


Info in <TCanvas::Print>: pdf file figs/egammEndcap_TkEle_EtaBC_rate.pdf has been created


In [68]:
# final_haset = []
labels_final = []

hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['EGBRL', 'TkEleELBRL'], 'LooseTkID', None)
for hist in hsets:
    hist.normalize(31000)

[his.h_pt_graph('online', 'rate [kHz]', convertRateToGraph) for his in hsets]
# final_haset.append(hsets[0])
# final_haset.append(hsets[0])

[his.g_pt_online.GetXaxis().SetLimits(0, 60) for his in hsets]
labels_final.append('calorimeter only')
labels_final.append('track-matched')

text_forprint='EG barrel, |#eta^{L1}| < 1.479'
draw([his.g_pt_online for his in hsets], labels_final, text=text_forprint, logy=True,
     min_x=0., max_x=60., min_y=1., max_y=75000, x_axis_label='threshold p_{T}^{L1} [GeV]',
     do_write=True, write_name='figs/egammaBarrel_TkEle_LooseTKID_rate'
    )


normalize to 31000
normalize to 31000


Info in <TCanvas::Print>: pdf file figs/egammaBarrel_TkEle_LooseTKID_rate.pdf has been created


In [69]:
labels_final = []
hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['EGBRL'], ['all', 'LooseTkID'], None)
for hist in hsets:
    hist.normalize(31000)

[his.h_pt_graph('online', 'rate [kHz]', convertRateToGraph) for his in hsets]
# final_haset.append(hsets[0])
# final_haset.append(hsets[0])

[his.g_pt_online.GetXaxis().SetLimits(0, 60) for his in hsets]
labels_final.append('w/o EG ID')
labels_final.append('w EG ID')

text_forprint='EG barrel, |#eta^{L1}| < 1.479'
draw([his.g_pt_online for his in hsets], labels_final, text=text_forprint, logy=True,
     min_x=0., max_x=60., min_y=1., max_y=75000, x_axis_label='threshold p_{T}^{L1} [GeV]',
     do_write=True, write_name='figs/EgammaBarrelOnlyCaloRate'
    )


normalize to 31000


Info in <TCanvas::Print>: pdf file figs/EgammaBarrelOnlyCaloRate.pdf has been created


In [70]:
# final_haset = []
labels_final = []

hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['TkEleELBRL'], ['EtaF'], None)
for hist in hsets:
    hist.normalize(31000)

[his.h_pt_graph('online', 'rate [kHz]', convertRateToGraph) for his in hsets]
# final_haset.append(hsets[0])
# final_haset.append(hsets[0])

[his.g_pt_online.GetXaxis().SetLimits(0, 60) for his in hsets]
# labels_final.append('calo only')
labels_final.append('track-matched')

text_forprint='<PU> = 200, |#eta^{L1}| #leq 1.47'
draw([his.g_pt_online for his in hsets], labels, text=text_forprint, logy=True,
     min_x=0., max_x=60., min_y=1., max_y=75000, x_axis_label='threshold p_{T}^{L1}[GeV]',
     do_write=True, write_name='egamma_TkEle_barrel_rate'
    )


Info in <TCanvas::Print>: pdf file egamma_TkEle_barrel_rate.pdf has been created


In [71]:
final_haset = []
labels_final = []
hsets, labels, text = hplot.get_histo(histos.RateHistos, 'old-ID', 'PU200', ['HMvDR'], 'EtaBC', None)
for hist in hsets:
    hist.normalize(31000)
    [his.h_pt_graph('online', 'rate [kHz]', convertRateToGraph) for his in hsets]

final_haset.append(hsets[0]) 
labels_final.append('w/o EG ID')

# hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['HMvDRCalib'], 'EmEtaBC', None)
# for hist in hsets:
#     hist.normalize(31000)
# final_haset.append(hsets[0]) 
# labels_final.append('EG ID clusters')

hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['EG'], 'EGq5EtaBC', None)
for hist in hsets:
    hist.normalize(31000)
[his.h_pt_graph('online', 'rate [kHz]', convertRateToGraph) for his in hsets]
[his.g_pt_online.GetXaxis().SetLimits(0, 60) for his in final_haset]

final_haset.append(hsets[0]) 

labels_final.append('w/ EG ID')
# print text
draw([his.g_pt_online for his in final_haset], labels_final, options='P', text='EG endcap, 1.52 < |#eta^{L1}| #leq 2.4', 
     logy=True, min_x=0., max_x=60., min_y=1., max_y=75000, x_axis_label='threshold p_{T}^{L1}[GeV]',
     do_write=True, write_name='figs/egammEndcap_EGID_EtaBC_rate')

No match found for: pu: PU200, tp: ['HMvDR'], tp_sel: EtaBC, gen_sel: None, classtype: python.l1THistos.RateHistos


TypeError: 'NoneType' object is not iterable

In [72]:
hsets, labels, text = hplot.get_histo(histos.RateHistos, sample, 'PU200', ['TkEle', 'EG'], 'EGq4EtaBC', None)
for hist in hsets:
    hist.normalize(31000)
draw([his.h_pt for his in hsets], labels=['EC standalone', 'EC + L1 Tk'], text=text, logy=True,
     do_write=True, write_name='egammEndcap_TkEle_EtaBC_rate')

Info in <TCanvas::Print>: pdf file egammEndcap_TkEle_EtaBC_rate.pdf has been created
